## Delcarações e Funções

In [119]:
!python -m spacy download pt_core_news_md

2022-11-28 19:45:27.935911: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.4 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')


In [120]:
# Load Python libraries
import io
import re
import spacy
from collections import Counter

In [121]:
# Util function to read a plain text file
def read_text_file(file_path):
    text = ""
    with io.open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

In [122]:
# Return an array with the clean words of the document
def get_doc_words(plain_text):
    doc_words = []
    
    # Cleaing the text
    clean_text = re.sub('[^-a-zA-ZÀ-ÖØ-öØ-ÿ]+|[.,!?;]', ' ', plain_text.lower())
    
    # Tokenize sentences in words
    doc_words = clean_text.split()
    
    return doc_words

In [123]:
# Get text sample
file_path = "corpus.txt"
plain_text = read_text_file(file_path)
len(plain_text)

5133765

In [124]:
# Create document word list
doc_words = get_doc_words(plain_text)
len(doc_words)

804504

In [125]:
# Show 100 first words of the vocabulary
print(doc_words[0:100])

['curiosidades', 'nos', 'últimos', 'séculos', 'os', 'avanços', 'na', 'área', 'da', 'medicina', 'foram', 'impressionantes', 'o', 'que', 'há', 'muitos', 'anos', 'parecia', 'impossível', 'hoje', 'já', 'é', 'realidade', 'como', 'os', 'transplantes', 'de', 'variados', 'órgãos', 'e', 'tecidos', 'os', 'avanços', 'nessa', 'área', 'foram', 'tão', 'surpreendentes', 'que', 'hoje', 'é', 'possível', 'até', 'mesmo', 'realizar', 'um', 'transplante', 'de', 'fezes', 'apesar', 'de', 'pouco', 'comum', 'o', 'transplante', 'de', 'fezes', 'é', 'realizado', 'por', 'algumas', 'pessoas', 'e', 'é', 'uma', 'alternativa', 'para', 'o', 'tratamento', 'de', 'alguns', 'tipos', 'de', 'diarreias', 'persistentes', 'conheça', 'a', 'seguir', 'um', 'pouco', 'mais', 'sobre', 'esse', 'processo', 'nada', 'convencional', 'o', 'transplante', 'de', 'fezes', 'ou', 'transplante', 'de', 'microbiota', 'fecal', 'o', 'transplante', 'de', 'fezes', 'também']


In [126]:
# Create term frequency list of the document words
term_freq = Counter(doc_words)
term_freq['que']

18512

In [127]:
# Define Spell Checker class
class SpellChecker:
    
    def __init__(self, term_freq):
        "Constructor."
        self.w_rank = {}
        self.letters = 'abcdefghijklmnopqrstuvwxyz'
        
        N = sum(term_freq.values())
        for term in term_freq:
            self.w_rank[term] = term_freq[term] / N
    
    def P(self, word): 
        "Probability of 'word'."
        return self.w_rank.get(word, 0)

    def known(self, words): 
        "The subset of 'words' that appear in the dictionary of w_rank."
        return set(w for w in words if w in self.w_rank)

    def edits1(self, word):
        "All edits that are one edit away from 'word'."
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in self.letters]
        inserts    = [L + c + R               for L, R in splits for c in self.letters]
        
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word): 
        "All edits that are two edits away from 'word'."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))
    
    def correction(self, word):
        "Most probable spelling correction for word."
        return max(self.candidates(word), key = self.P)
    
    def candidates(self, word): 
        "Generate possible spelling corrections for word."
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

In [128]:
# Create model
sp_model = SpellChecker(term_freq)

In [160]:
# Get probability of the word 'the'
sp_model.P('árvore')

0.00010068315384385908

In [130]:
# Get words that exist in the dictionary
sp_model.known(['trabalhar', 'não-mencionado'])

{'trabalhar'}

In [131]:
# Get the correction for word 'castli'
sp_model.correction('traalo')

'trabalho'

In [158]:
# Get candidates for word 'wlak'
sp_model.candidates('arvore')

{'arborea', 'ardor'}

## **Executáveis**




In [177]:
textoParaAnalise = 'qur abaçar um árvre?'

In [178]:
#Recebe uma string com o texto a ser corrigido, e o corrige Ortográficamente.


frase = get_doc_words(textoParaAnalise)
fraseCorrigida = ''

for palavra in frase:
  fraseCorrigida += sp_model.correction(palavra) + ' '

print(fraseCorrigida)

que abraçar um árvore 


In [180]:
#Checa se o plural das palavras e o gênero delas concordam.

nlp = spacy.load("pt_core_news_md")
doc = nlp(fraseCorrigida)
print('Frase: ' + doc.text)
for token in doc:
    if(token.morph.get('Gender') != []):
      if(token.morph.get('Gender') != last.morph.get('Gender') and last.morph.get('Gender') != []):
        print('-> ' + last.text +' <-'+ ' \nEstá errado. Gênero da próxima palavra não concorda com -> '+ token.text +' <-.')
    
      if(token.morph.get('Number') != []):
        if(token.morph.get('Number') != last.morph.get('Number') and last.morph.get('Number') != []):
          print('-> ' + last.text +' <-'+ ' \nEstá errado. Plural da próxima palavra não concorda com -> '+ token.text +' <-.')
    last = token

Frase: que abraçar um árvore 
-> um <- 
Está errado. Gênero da próxima palavra não concorda com -> árvore <-.
